In [1]:
## mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install snntorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 9.9 MB/s eta 0:00:00


In [3]:
## os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
import gc
import snntorch as snn
from snntorch import surrogate
from snntorch import utils

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "base_dir": "/content/drive/MyDrive/GlacierHack_practice/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_Project",
    "model_type": "SNN",

    "time_steps": 8,       # T=8 for better resolution
    "batch_size": 1,       # Keep 1 for safety
    "base_filters": 32,    # Increased back to 32 for better feature detection

    "beta": 0.7,           # Slightly higher retention for T=8
    "threshold": 1.0,
    "epochs": 50,
    "lr": 1e-3,
    "num_workers": 2,
    "seed": 42,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

torch.cuda.empty_cache()
gc.collect()
os.makedirs(CONFIG['project_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(CONFIG['seed'])

# ==========================================
# 2. DATASET & AUGMENTATION
# ==========================================
class GlacierDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = Path(base_dir)
        self.band_dirs = [self.base_dir / f"Band{i}" for i in range(1, 6)]
        self.label_dir = self.base_dir / "labels"
        self.ids = sorted([p.stem for p in self.band_dirs[0].glob("*.tif")])
        self.transform = transform

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        bands = [cv2.imread(str(d / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED).astype(np.float32) for d in self.band_dirs]
        image = np.stack(bands, axis=-1)
        label = cv2.imread(str(self.label_dir / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED)
        if label.ndim == 3: label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        mask = np.zeros_like(label, dtype=np.uint8)
        mask[label == 85] = 1; mask[label == 170] = 2; mask[label == 255] = 3

        p02, p98 = np.percentile(image, 2), np.percentile(image, 98)
        image = np.clip(image, p02, p98)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug["image"].float(), aug["mask"].long()
        return torch.tensor(image.transpose(2,0,1)).float(), torch.tensor(mask).long()

class Wrapper(Dataset):
    def __init__(self, ds, t): self.ds, self.t = ds, t
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        img, mask = self.ds[i]
        img = img.numpy().transpose(1,2,0); mask = mask.numpy()
        res = self.t(image=img, mask=mask)
        return res['image'], res['mask'].long()

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5),
    A.GridDistortion(p=0.3),
    ToTensorV2(),
])
val_transform = A.Compose([ToTensorV2()])

full_dataset = GlacierDataset(CONFIG['base_dir'], transform=None)
val_len = int(len(full_dataset)*0.2)
train_ds, val_ds = random_split(full_dataset, [len(full_dataset)-val_len, val_len])

train_loader = DataLoader(Wrapper(train_ds, train_transform), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(Wrapper(val_ds, val_transform), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

# ==========================================
# 3. ARCHITECTURE (Standard Width)
# ==========================================
class SNNBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        spike_grad = surrogate.atan()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'],
                      spike_grad=spike_grad, init_hidden=True,
                      learn_beta=True, learn_threshold=True),

            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'],
                      spike_grad=spike_grad, init_hidden=True,
                      learn_beta=True, learn_threshold=True)
        )
    def forward(self, x): return self.net(x)

class ProjectUNet(nn.Module):
    def __init__(self, in_ch=5, n_classes=4, f=32): # f=32 Standard Width
        super().__init__()
        Block = SNNBlock
        self.inc = Block(in_ch, f)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), Block(f, f*2))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), Block(f*2, f*4))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), Block(f*4, f*8))

        self.up1 = nn.ConvTranspose2d(f*8, f*4, kernel_size=2, stride=2)
        self.conv1 = Block(f*8, f*4)
        self.up2 = nn.ConvTranspose2d(f*4, f*2, kernel_size=2, stride=2)
        self.conv2 = Block(f*4, f*2)
        self.up3 = nn.ConvTranspose2d(f*2, f, kernel_size=2, stride=2)
        self.conv3 = Block(f*2, f)
        self.outc = nn.Conv2d(f, n_classes, 1)

    def forward(self, x):
        spk_rec = []
        for step in range(CONFIG['time_steps']):
            x1 = self.inc(x)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)

            x_up1 = self.up1(x4)
            if x_up1.shape != x3.shape: x_up1 = F.interpolate(x_up1, size=x3.shape[2:])
            x5 = self.conv1(torch.cat([x3, x_up1], dim=1))

            x_up2 = self.up2(x5)
            if x_up2.shape != x2.shape: x_up2 = F.interpolate(x_up2, size=x2.shape[2:])
            x6 = self.conv2(torch.cat([x2, x_up2], dim=1))

            x_up3 = self.up3(x6)
            if x_up3.shape != x1.shape: x_up3 = F.interpolate(x_up3, size=x1.shape[2:])
            x7 = self.conv3(torch.cat([x1, x_up3], dim=1))

            out = self.outc(x7)
            spk_rec.append(out)

        return torch.stack(spk_rec) # Return [T, B, C, H, W]

# ==========================================
# 4. UTILS (Manual Reset + Vis)
# ==========================================
def manual_reset(model):
    for m in model.modules():
        if hasattr(m, "reset_mem"): m.reset_mem()

def save_plot_and_sample(history, sample_data, epoch):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.plot(history['loss'], label='Train Loss')
    plt.title("SNN Loss")
    plt.legend()
    plt.subplot(1, 3, 2)
    plt.plot(history['mcc'], label='Val MCC', color='green')
    plt.title("SNN MCC")
    plt.legend()
    plt.subplot(1, 3, 3)
    plt.plot(history['sparsity'], label='Spike Rate', color='orange')
    plt.title("Sparsity")
    plt.legend()
    plt.savefig(f"{CONFIG['project_dir']}/SNN_Weighted_metrics.png")
    plt.close()

    img, gt, pred = sample_data
    rgb = img[[3,2,1]].transpose(1,2,0)
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-6)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(rgb); plt.title("Input")
    plt.subplot(1, 3, 2); plt.imshow(gt, cmap='nipy_spectral'); plt.title("Ground Truth")
    plt.subplot(1, 3, 3); plt.imshow(pred, cmap='nipy_spectral'); plt.title(f"SNN Pred (Ep {epoch})")
    plt.savefig(f"{CONFIG['project_dir']}/SNN_Weighted_sample.png")
    plt.close()

# ==========================================
# 5. TRAINING LOOP (With Weights)
# ==========================================
print(f"🚀 Initializing SNN (Filters={CONFIG['base_filters']})...")
model = ProjectUNet(in_ch=5, n_classes=4, f=CONFIG['base_filters']).to(CONFIG['device'])
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])

# --- CRITICAL: WEIGHTED LOSS ---
# [Background, Debris, Ice, Snow]
# We heavily penalize missing Snow (Class 3)
weights = torch.tensor([0.2, 1.0, 1.0, 3.0]).to(CONFIG['device'])
criterion = nn.CrossEntropyLoss(weight=weights)

best_mcc = -1.0
history = {'loss': [], 'mcc': [], 'sparsity': []}

print(f"🔥 Starting SNN Training (Weighted Loss)...")

for epoch in range(CONFIG['epochs']):
    model.train()
    running_loss = 0

    pbar = tqdm(train_loader, desc=f"Ep {epoch+1}")
    for imgs, masks in pbar:
        imgs, masks = imgs.to(CONFIG['device']), masks.to(CONFIG['device']).long()
        manual_reset(model)
        optimizer.zero_grad()

        spk_out = model(imgs)
        mean_out = spk_out.mean(0)

        loss = criterion(mean_out, masks)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})

    avg_loss = running_loss / len(train_loader)
    history['loss'].append(avg_loss)

    # Validation
    model.eval()
    preds, targets = [], []
    sample_vis = None
    total_spikes, total_elements = 0, 0

    with torch.no_grad():
        for i, (imgs, masks) in enumerate(val_loader):
            imgs = imgs.to(CONFIG['device'])
            manual_reset(model)

            spk_out = model(imgs)
            mean_out = spk_out.mean(0)

            total_spikes += (spk_out > 0).float().sum().item()
            total_elements += spk_out.numel()

            p = mean_out.argmax(1).cpu()
            preds.append(p)
            targets.append(masks.cpu())

            if i == 0:
                sample_vis = (imgs[0].cpu().numpy(), masks[0].cpu().numpy(), p[0].numpy())

    mcc = matthews_corrcoef(torch.cat(targets).numpy().flatten(), torch.cat(preds).numpy().flatten())
    spike_rate = total_spikes / (total_elements + 1e-6)

    history['mcc'].append(mcc)
    history['sparsity'].append(spike_rate)

    print(f"Ep {epoch+1} | Loss: {avg_loss:.4f} | Val MCC: {mcc:.4f} | Spike Rate: {spike_rate:.4f}")

    save_plot_and_sample(history, sample_vis, epoch+1)

    if mcc > best_mcc:
        best_mcc = mcc
        torch.save(model.state_dict(), f"{CONFIG['project_dir']}/best_SNN_Weighted.pth")
        print("✅ Saved Best SNN Model!")

print(f"🏁 Training Complete. Best MCC: {best_mcc:.4f}")

🚀 Initializing SNN (Filters=32)...
🔥 Starting SNN Training (Weighted Loss)...


Ep 1: 100%|██████████| 20/20 [01:11<00:00,  3.57s/it, loss=0.982]


Ep 1 | Loss: 1.1979 | Val MCC: 0.4316 | Spike Rate: 0.4478
✅ Saved Best SNN Model!


Ep 2: 100%|██████████| 20/20 [00:18<00:00,  1.07it/s, loss=0.911]


Ep 2 | Loss: 0.9892 | Val MCC: 0.4490 | Spike Rate: 0.3810
✅ Saved Best SNN Model!


...

Ep 48 | Loss: 0.5548 | Val MCC: 0.4948 | Spike Rate: 0.3554


Ep 49: 100%|██████████| 20/20 [00:19<00:00,  1.03it/s, loss=0.402]


Ep 49 | Loss: 0.5550 | Val MCC: 0.5283 | Spike Rate: 0.3719
✅ Saved Best SNN Model!


Ep 50: 100%|██████████| 20/20 [00:19<00:00,  1.03it/s, loss=0.446]


Ep 50 | Loss: 0.5406 | Val MCC: 0.4345 | Spike Rate: 0.3286
🏁 Training Complete. Best MCC: 0.5283
